# Integer Programming - Vehicle Routing Problem (VRP)
Yang Xi <br>
14 Aug, 2018

<br>

- Problem Statement
    - Vehicle Routing Problem (VRP)
    - VRP Optimization
- Integer Programming
    - Variables
    - Objective
    - Constraints
    - Sub-tour Avoidance
- Implementation
    - Helper Functions
    - Test Case: Sample Problem
    - Test Case: Sub-Tour Avoidance
    - Test Case: Load Constraint

# Problem Statement

In this notebook, I will demonstrate how to use Integer Programming to solve an optimization problem.

<br>

### Vehicle Routing Problem (VRP)
Every day, a delivery company need to deliver goods from its central warehouse to many customers, using a fleet of vehicles.<br>
The goal is to design a route for each vehicle so that
* all customers are served by exactly one vehicle
* total travel distance of the vehicles is minimized

The vehicles have fixed capacity and customers have different demands.<br>
Figure below illustrates a sample VRP and a feasible (but not optimal) solution to the problem:
<img src="images/sample VRP.jpg">

The locations are labeled from 0 to 4, with 0 being the warehouse, and 1 to 4 being the customers. The solution uses two vehicles with are indicated by different colored routes.

<br>

### VRP Optimization
List of $N$ locations $L_n$, where $n=0...N-1$ :
* $L_0$ is the warehouse. All vehicles start and end their routes at the warehouse.
* Remaining locations are the customers.
* $D[j,k]$ is the **Euclidean distance** between $L_j$ and $L_k$
* Here we assume vehicles can travel in straight lines.

Each location is characterized by three values $<d_n, x_n, y_n>$ :
* demand $dn$ at point $x_n, y_n$, with $1\leq n \leq N-1$

Fleet of $V$ vehicles $H_v$, where $v=0...V-1$ :
* each vehicle has a limited capacity $c$.

Let
* $T_v$ be a list with the sequence of deliveries made by vehicle $v$, excluding the warehouse
    * Each element $T_{v,i}$ is one location, with $i=0...|T_v|-1$
    * $T_{v,i}\in 1...N-1$

The VRP is formulated as the following optimization problem:<br>
<br>
*minimize:*
$$ \sum\limits_{\substack{v\in V}} \left( D[0,T_{v,0}]+\sum_{j=1}^{|T_v|-2}D[T_{v,j},T_{v,j+1}]+D[T_{v,|T_v|-1},0] \right) $$
*subject to:*
$$ \sum\limits_{\substack{j\in T_v}}d_j \leq c \space (v\in V) $$
$$ \sum\limits_{\substack{v\in V}}(l\in T_v)=1 \space (l\in 1...N-1) $$


# Integer Programming
The VRP optimization problem can be formulated as an Interger Programming model in the following manner:<br>

<br>

### Variables
$x[v,a,b]$ : Whether vehicle $v$ will travel from $L_a$ to $L_b$
$$ x[v,a,b]\in[0,1] $$

<br>

### Objective
$$ minimize\sum_v\sum_a\sum_b x[v,a,b]D[a,b] $$

<br>

### Constraints


**Location Constraints**<br>
These contraints apply to each location.<br>
(customers) Each customer is visited by one and only one vehicle, one and only once:
$$ \sum_v\sum_b x[v,a,b]=1,\text{ for }a=1..N-1 $$
$$ \sum_v\sum_a x[v,a,b]=1,\text{ for }b=1..N-1 $$
<br>
(warehouse) The warehouse always have the same number of arrivals and departures:
$$ \sum_v\sum_b x[v,0,b]=\sum_v\sum_b x[v,b,0] $$

<br>

**Vehicle Constraints**<br>
These contraints apply to each vehicle.<br>
(load) A vehicle cannot carry more then it's capacity:
$$ \sum_a\sum_b x[v,a,b]\frac{d_a+d_b}{2}\leq c,\text{ for }v=0...V-1 $$
<br>
(start point) A vehicle always starts from warehouse if assigned:<br>
*Let:*
$$ A_v\text{(is assigned)}=\sum_a\sum_b x[v,a,b] $$
$$ F_v\text{(is from warehouse)}=\sum_b x[v,a,b] $$
$$ M\text{(a large multiplier)} = 10000 $$
*Constraints:*
$$ A_v-F_v\geq 0,\text{ for }v=0...V-1 $$
$$ A_v\leq MF_v,\text{ for }v=0...V-1 $$

<br>

**Flow Constratins**<br>
These contraints apply to each vehicle at each location.<br>
(flow) If a vehicle (not) arrives in a location, it must also (not) depart from that location
$$ \sum_b x[v,a,b]=\sum_b x[v,b,a],\text{ for }v=0...V-1,a=1...N-1 $$
<br>
(no loop) No loop for each vehicle at each customer:
$$ x[v,a,a]==0,\text{ for }v=0...V-1,a=1...N-1 $$

<br>

### Sub-tour Avoidance
Sub-tour avoidance is implemented by adding lazy constraints in iterations:
1. Solve the optimization problem 
2. If no feasible solution: 
    * Break the loop 
    * Output "no feasible solution"
3. Exam the feasible solution for sub-tour 
4. If no sub-tour: 
    * Break the loop 
    * Output the optimized solution 
5. Add the identified sub-tour to constraints 
6. Restart from step 1 

# Implementation
### Helper Functions
The following helper functions are defined to implement the Integer Programming model.

In [1]:
import pandas as pd
from scipy.spatial import distance_matrix
from pulp import *

# Function to define and solve the Integer Programming model
def DefineSolveIP(N,V,c,d,x,y):
    assert len(d) == len(x), "length of location characters (d,x,y) should be the same length"
    assert len(x) == len(y), "length of location characters (d,x,y) should be the same length"
    assert N == len(d), "Number of locations (N) should match length of location characters (d,x,y)"
    
    v_N = [n for n in range(N)]
    v_V = [v for v in range(V)]
    m_D = distance_matrix(list(zip(x,y)), list(zip(x,y)))

    # Define optimization problem
    prob = LpProblem(name='VRP', sense=LpMinimize)

    # Define binary variables
    x_vab = LpVariable.dict('xvab',(v_V, v_N, v_N), 0, 1, LpInteger) # vehicle v, location a, location b

    # Define objective
    prob += lpSum([x_vab[v,a,b]*m_D[a,b] for v in v_V for a in v_N for b in v_N]), 'cost'

    # Constraints
    # Location constraints
    for a in v_N:
        if a==0: # warehouse
            prob += lpSum([x_vab[v,0,b] for v in v_V for b in v_N]) == lpSum([x_vab[v,b,0] for v in v_V for b in v_N])
        else: # customers
            prob += lpSum([x_vab[v,a,b] for v in v_V for b in v_N]) == 1
            prob += lpSum([x_vab[v,b,a] for v in v_V for b in v_N]) == 1

    # Vehicle constraints
    for v in v_V:            
        # load
        prob += lpSum([x_vab[v,a,b]*(d[a]+d[b])/2 for a in v_N for b in v_N]) <= c
        # start point
        Assign = lpSum([x_vab[v,a,b] for a in v_N for b in v_N])
        Fware = lpSum([x_vab[v,0,b] for b in v_N])
        prob += (Assign-Fware) >= 0
        prob += Assign <= (10000*Fware)

    # Flow constraints
    for a in v_N:
        for v in v_V:
            prob += lpSum([x_vab[v,a,b] for b in v_N]) == lpSum([x_vab[v,b,a] for b in v_N]) # flow
            prob += x_vab[v,a,a] == 0 # no loop
    
    is_solved = prob.solve()==1
    return x_vab, prob, is_solved

In [2]:
# Function to extract results from the optimized solution
def ExtractResult(prob):
    lVehicleRoutes = [xvab.name for xvab in prob.variables() if xvab.varValue>0 and ('y' not in xvab.name)]
    dfVehicleRoutes = pd.DataFrame(lVehicleRoutes, columns=['xvab'])
    dfVehicleRoutes['v'] = dfVehicleRoutes['xvab'].replace('xvab_|_[0-9]+_[0-9]+','',regex=True)
    
    def StrToList(df):
        seA = df['xvab'].replace('xvab_[0-9]+_|_[0-9]+$','',regex=True).astype(int)
        seB = df['xvab'].replace('xvab_[0-9]+_[0-9]+_','',regex=True).astype(int)
        return list(zip(seA,seB))
    
    return dfVehicleRoutes.groupby('v').apply(StrToList).rename('ab').reset_index()

In [3]:
# Function to check and remove subtour once
def CheckSubTour(prob, x_vab, V):
    # lRoutes is a list of routes of each vehicle, like [[(0,2),(2,0),(3,4),(4,3)],[(0,1),(1,0)]]
    lRoutes = ExtractResult(prob)['ab']
    
    is_subtour = False # reset subtour flag
    is_solved = True
    for lAB in lRoutes: 
        seRoute = pd.Series(lAB).copy()
        # start from warehouse and follow the route.
        # If the route hasn't been completed but the sequence is broken, there must be subtour.
        s = 0
        for k in range(len(lAB)):
            indexStart = seRoute.apply(lambda l:l[0]==s)
            if indexStart.any():
                lCurrentRoute = seRoute[indexStart].iloc[0]
                seRoute = seRoute[seRoute.apply(lambda l:l!=lCurrentRoute)]
                s = lCurrentRoute[1]
            else: # the remaining routes are subtour
                is_subtour = True
                print(f'subtour detected and removed:{seRoute.tolist()}')
                # add subtour constraints for each vehicle
                for v in range(V):
                    prob += lpSum([x_vab[v,a,b] for a, b in seRoute.tolist()]) <= (len(seRoute)-1)
                break;
        if is_subtour:
            is_solved = prob.solve()==1
    return x_vab, prob, is_subtour, is_solved


In [4]:
# Function to check and remove all subtours recursively
def RemoveSubTours(prob, x_vab, V):
    is_subtour = True
    is_solved = True
    while(is_subtour and is_solved):
        x_vab, prob, is_subtour, is_solved = CheckSubTour(prob, x_vab, V)
    return x_vab, prob, is_subtour, is_solved


In [5]:
# Function sort the routes in sequence
def SortRoute(lAB):
    seRoute = pd.Series(lAB).copy()
    s, route_sorted = 0, [0] # start from warehouse
    for k in range(len(lAB)):
        indexStart = seRoute.apply(lambda l:l[0]==s)
        lCurrentRoute = seRoute[indexStart].iloc[0]
        seRoute = seRoute[seRoute.apply(lambda l:l!=lCurrentRoute)]
        s = lCurrentRoute[1]
        route_sorted.append(s)

    return route_sorted


### Test Case: Sample Problem
Finally, let's use the defined functions to solve the sample problem!
<img src="images/sample VRP.jpg">

In [6]:
dInputs = {'N':5, 'V':4, 'c':10,
           'd':[0, 3,  3,  3,  3],
           'x':[0, 0,-10,  0, 10],
           'y':[0,10, 10,-10,-10]}

x_vab, prob, is_solved = DefineSolveIP(**dInputs)
if is_solved:
    x_vab, prob, is_subtour, is_solved = RemoveSubTours(prob, x_vab, dInputs['V'])
if is_solved:
    dfVehicleRoutes = ExtractResult(prob)
    dfVehicleRoutes['route_sorted'] = dfVehicleRoutes['ab'].apply(SortRoute)
    display(dfVehicleRoutes)
    print(f'Optimized cost = {value(prob.objective)}')
else:
    print('No feasible solution.')

,v,ab,route_sorted
0,1,"[(0, 2), (1, 0), (2, 1)]","[0, 2, 1, 0]"
1,2,"[(0, 3), (3, 4), (4, 0)]","[0, 3, 4, 0]"


Optimized cost = 68.2842712474619


**Awesome!** The optimized solution has lower cost than the feasible solution in the figure above.

<br>

In the following sections, I will carry out more tests.

### Test Case: Sub-Tour Avoidance
This is a simple test case with 1 vehicle visiting multiple locations.<br>
The output displays that the algorithm managed to detect sub-tour from an "optimal" solution, and generate a true optimal solution with the sub-tour avoided.

In [7]:
dInputs = {'N':4, 'V':1, 'c':10,
           'd':[0, 3, 3, 3],
           'x':[0, 0,10,20],
           'y':[0,10, 0, 0]}

x_vab, prob, is_solved = DefineSolveIP(**dInputs)
if is_solved:
    x_vab, prob, is_subtour, is_solved = RemoveSubTours(prob, x_vab, dInputs['V'])
if is_solved:
    dfVehicleRoutes = ExtractResult(prob)
    dfVehicleRoutes['route_sorted'] = dfVehicleRoutes['ab'].apply(SortRoute)
    display(dfVehicleRoutes)
    print(f'Optimized cost = {value(prob.objective)}')
else:
    print('No feasible solution.')

subtour detected and removed:[(2, 3), (3, 2)]


,v,ab,route_sorted
0,0,"[(0, 1), (1, 3), (2, 0), (3, 2)]","[0, 1, 3, 2, 0]"


Optimized cost = 52.3606797749979


### Test Case: Load Constraint
In the test case, one vehicle cannot complete the task by visiting the warehouse only once.<br>
It seems that $L_2$ and $L_3$ could form a sub-tour, while it will be limited by the load constraint, because the total load of a vehicle is NOT calculated for each sub-tour.

In [8]:
dInputs = {'N':4, 'V':1, 'c':10,
           'd':[0,10, 5, 5],
           'x':[0, 0,10,20],
           'y':[0,10, 0, 0]}

x_vab, prob, is_solved = DefineSolveIP(**dInputs)
if is_solved:
    x_vab, prob, is_subtour, is_solved = RemoveSubTours(prob, x_vab, dInputs['V'])
if is_solved:
    dfVehicleRoutes = ExtractResult(prob)
    dfVehicleRoutes['route_sorted'] = dfVehicleRoutes['ab'].apply(SortRoute)
    display(dfVehicleRoutes)
    print(f'Optimized cost = {value(prob.objective)}')
else:
    print('No feasible solution.')

No feasible solution.


With 2 vehicles, we can complete the task:

In [9]:
dInputs = {'N':4, 'V':2, 'c':10,
           'd':[0,10, 5, 5],
           'x':[0, 0,10,20],
           'y':[0,10, 0, 0]}

x_vab, prob, is_solved = DefineSolveIP(**dInputs)
if is_solved:
    x_vab, prob, is_subtour, is_solved = RemoveSubTours(prob, x_vab, dInputs['V'])
if is_solved:
    dfVehicleRoutes = ExtractResult(prob)
    dfVehicleRoutes['route_sorted'] = dfVehicleRoutes['ab'].apply(SortRoute)
    display(dfVehicleRoutes)
    print(f'Optimized cost = {value(prob.objective)}')
else:
    print('No feasible solution.')

,v,ab,route_sorted
0,0,"[(0, 2), (2, 3), (3, 0)]","[0, 2, 3, 0]"
1,1,"[(0, 1), (1, 0)]","[0, 1, 0]"


Optimized cost = 60.0
